# Описание проекта 

**Данные**

База данных с комментариями пользователей и отметкой о токсичности комментария.

**Задача**

Обучить модель классифицировать комментарии на позитивные и негативные со значением метрики качества *F1* не меньше 0.75. 

**Инструкция**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# Подготовка

In [64]:
import pandas as pd
import numpy as np

import warnings

from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from lightgbm import LGBMClassifier

In [65]:
data = pd.read_csv('/datasets/toxic_comments.csv')
# data = pd.read_csv("C:/Users/kbolo/Documents/Python Scripts/Yandex/Project 12 - ML - токсичные комменты/toxic_comments.csv")

In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [67]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Пропусков нет

In [68]:
# Дубликатов нет
data.duplicated().sum()

0

In [69]:
data.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Классы несбалансированны.  
Учту в параметре модели -- class_weight

## Лемматизация (стемминг)

In [11]:
stop_words = stopwords.words('english')

In [12]:
# Кастомная функция для стемминга
def stemm(text):
    text = word_tokenize(text.lower())
    stemmer = LancasterStemmer()
    text = [stemmer.stem(w) for w in text if w not in stop_words and w.isalpha()]
    return ' '.join(text)

In [14]:
tqdm.pandas()
features = data.text.progress_apply(stemm)

100%|█████████████████████████████████████████████████████████████████████████| 159292/159292 [02:49<00:00, 937.89it/s]


In [39]:
features.head()

0    expl edit mad usernam hardc metallic fan rever...
1    match background colo seem stuck thank talk ja...
2    hey man real try edit war guy const remov rele...
3    ca mak real suggest improv wond sect stat lat ...
4                            sir hero chant rememb pag
Name: text, dtype: object

## Деление на выборки

In [38]:
X_train, X_test, y_train, y_test = train_test_split(
                features, data.toxic, test_size=0.1, random_state=123)

# Обучение

## LogisticRegression

In [21]:
pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('model', LogisticRegression(random_state=123))])
params = {'model__class_weight': [None, 'balanced']
         }

gridsearch = GridSearchCV(pipeline, params, scoring='f1', n_jobs=-1)
gridsearch.fit(X_train, y_train);

In [22]:
pd.DataFrame(gridsearch.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,{'model__class_weight': None},0.740502
1,{'model__class_weight': 'balanced'},0.734494


In [23]:
pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('model', LogisticRegression(random_state=123))])
params = {'model__C': [6, 8, 10],
          'model__tol': [0.01, 0.05, 0.1],
          'model__solver': ['lbfgs', 'saga'],
         }

gridsearch = GridSearchCV(pipeline, params, scoring='f1', n_jobs=-1)
gridsearch.fit(X_train, y_train);

In [25]:
model_LR = gridsearch

In [36]:
model_LR.best_params_

{'model__C': 8, 'model__solver': 'saga', 'model__tol': 0.05}

## LGBMClassifier

In [27]:
# Гиперпараметры по умолчанию
pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('model', LGBMClassifier(random_state=123))])
params = {}

gridsearch = GridSearchCV(pipeline, params, scoring='f1', n_jobs=-1)
gridsearch.fit(X_train, y_train);

In [28]:
gridsearch.best_score_

0.7476910724456113

In [29]:
# Кастомные гиперпараметры
pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('model', LGBMClassifier(random_state=123))])
params = {'model__max_depth': [2,3],
          'model__n_estimators': [100,200],
          'model__learning_rate':[0.2, 0.3]
         }

gridsearch = GridSearchCV(pipeline, params, scoring='f1', n_jobs=-1)
gridsearch.fit(X_train, y_train);

In [30]:
gridsearch.best_score_

0.7331567804532357

# Тестирование

Для тестирования беру LinearClassifier, т.к. показала лучший результат

In [37]:
model_LR.score(X_test, y_test) 

0.7794117647058824

# Выводы

* Цель F1 >= 0.75 достигнута
* LinearClassifier показала себя лучше LGBMClassifier